Import Required Libraries

In [1]:
from google.cloud import bigquery
import pandas as pd
import pandas_gbq
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

Set Environment variable for Google Credentials

In [2]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/karthikeyangurusamy/Documents/GCP/key.json"

Create Big Query Client

In [3]:
bg_client = bigquery.Client(project='bigquery-public-data')

Get the data set and display all the tables from the given dataset

In [4]:
data_set_ref = bg_client.dataset('chicago_taxi_trips', project='bigquery-public-data')
data_set = bg_client.get_dataset(data_set_ref)
for tab in bg_client.list_tables(data_set):
    print(tab.table_id)

taxi_trips


List all the columns and the corresponding details for the given table

In [5]:
tab = bg_client.get_table(data_set.table('taxi_trips'))
tab.schema

[SchemaField('unique_key', 'STRING', 'REQUIRED', 'Unique identifier for the trip.', (), None),
 SchemaField('taxi_id', 'STRING', 'REQUIRED', 'A unique identifier for the taxi.', (), None),
 SchemaField('trip_start_timestamp', 'TIMESTAMP', 'NULLABLE', 'When the trip started, rounded to the nearest 15 minutes.', (), None),
 SchemaField('trip_end_timestamp', 'TIMESTAMP', 'NULLABLE', 'When the trip ended, rounded to the nearest 15 minutes.', (), None),
 SchemaField('trip_seconds', 'INTEGER', 'NULLABLE', 'Time of the trip in seconds.', (), None),
 SchemaField('trip_miles', 'FLOAT', 'NULLABLE', 'Distance of the trip in miles.', (), None),
 SchemaField('pickup_census_tract', 'INTEGER', 'NULLABLE', 'The Census Tract where the trip began. For privacy, this Census Tract is not shown for some trips.', (), None),
 SchemaField('dropoff_census_tract', 'INTEGER', 'NULLABLE', 'The Census Tract where the trip ended. For privacy, this Census Tract is not shown for some trips.', (), None),
 SchemaField('

In [6]:
tab.num_rows

194212416

Create a Sample query and assign the data into Pandas Dataframe

In [7]:
QUERY = """
    SELECT EXTRACT(YEAR FROM trip_start_timestamp) AS YEAR, COUNT(unique_key) as TRIPS
    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
    GROUP BY YEAR
        """
df=pd.read_gbq(QUERY)
df

Downloading: 100%|██████████| 8/8 [00:00<00:00, 21.22rows/s]


,YEAR,TRIPS
0,2019,16477365
1,2013,27217716
2,2017,24988003
3,2015,32385875
4,2014,37395436
5,2016,31759339
6,2018,20732088
7,2020,3256594


# Get Data

Lets take the data corresponding to Feb-2020 to build a base model

In [8]:
QUERY = """
    SELECT *
    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
    WHERE EXTRACT(YEAR FROM trip_start_timestamp) = 2020
    AND EXTRACT(MONTH FROM trip_start_timestamp) = 2
    """

df=pd.read_gbq(QUERY)
df.head(5)

Downloading: 100%|██████████| 1122124/1122124 [04:27<00:00, 4187.36rows/s]


,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,ba8e167fd614d76bec89951efc3c2439e321e767,f1ed8794238deb1c8f7c3759aa096ae20e3e769cc4c9ad...,2020-02-01 00:00:00+00:00,2020-02-01 00:15:00+00:00,593.0,2.8,NaN,NaN,76.0,NaN,...,4.0,17.70,Credit Card,Metro Jet Taxi A,41.980264,-87.913625,POINT (-87.913624596 41.9802643146),NaN,NaN,None
1,70ad9e34b0a373e0bed335bf92552548d2a9b6ee,e45877a53e727ebec449c17f0835d75f2f527435559fd9...,2020-02-01 00:00:00+00:00,2020-02-01 00:00:00+00:00,780.0,3.6,NaN,NaN,8.0,24.0,...,1.0,13.75,Cash,Taxi Affiliation Services,41.899602,-87.633308,POINT (-87.6333080367 41.899602111),41.901207,-87.676356,POINT (-87.6763559892 41.9012069941)
2,4e5173d8c0b558afba1d488e4679d90ad4eda307,de3382b499da313268a07eae4c95991b625d16a41af1ff...,2020-02-01 00:00:00+00:00,2020-02-01 00:00:00+00:00,420.0,0.0,NaN,NaN,8.0,32.0,...,1.0,8.50,Cash,Taxi Affiliation Services,41.899602,-87.633308,POINT (-87.6333080367 41.899602111),41.878866,-87.625192,POINT (-87.6251921424 41.8788655841)
3,af1e6f3a4b3b0a2d981fd6493d80fcc3fd6f0ffd,51d13e9ec1a04be36d87a3746e0e4f8c68ca5ed59e6f7a...,2020-02-01 00:00:00+00:00,2020-02-01 00:00:00+00:00,710.0,1.8,NaN,NaN,8.0,8.0,...,1.0,9.75,Cash,City Service,41.899602,-87.633308,POINT (-87.6333080367 41.899602111),41.899602,-87.633308,POINT (-87.6333080367 41.899602111)
4,c085286c289f4d2d20b6991bff839875aabb74bd,d4578429775c32e03fb5139e1d132b91be7e27af9675e7...,2020-02-01 00:00:00+00:00,2020-02-01 00:15:00+00:00,600.0,1.4,NaN,NaN,28.0,8.0,...,0.0,10.00,Credit Card,Taxi Affiliation Services,41.874005,-87.663518,POINT (-87.6635175498 41.874005383),41.899602,-87.633308,POINT (-87.6333080367 41.899602111)


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1122124 entries, 0 to 1122123
Data columns (total 23 columns):
 #   Column                  Non-Null Count    Dtype              
---  ------                  --------------    -----              
 0   unique_key              1122124 non-null  object             
 1   taxi_id                 1122124 non-null  object             
 2   trip_start_timestamp    1122124 non-null  datetime64[ns, UTC]
 3   trip_end_timestamp      1122076 non-null  datetime64[ns, UTC]
 4   trip_seconds            1121943 non-null  float64            
 5   trip_miles              1122118 non-null  float64            
 6   pickup_census_tract     711601 non-null   float64            
 7   dropoff_census_tract    709129 non-null   float64            
 8   pickup_community_area   1042701 non-null  float64            
 9   dropoff_community_area  1021866 non-null  float64            
 10  fare                    1122014 non-null  float64            
 11  tips       

For Model purpose, lets choose only the below fields
1. trip_start_timestamp
2. pickup_latitude, pickup_longitude
3. dropoff_latitude, dropoff_longitude
4. compare
5. fare - This field will be our label to predict

In [10]:
df_for_model = df[['trip_start_timestamp','pickup_latitude','pickup_longitude','dropoff_latitude','dropoff_longitude','company','fare']]
df_for_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1122124 entries, 0 to 1122123
Data columns (total 7 columns):
 #   Column                Non-Null Count    Dtype              
---  ------                --------------    -----              
 0   trip_start_timestamp  1122124 non-null  datetime64[ns, UTC]
 1   pickup_latitude       1042806 non-null  float64            
 2   pickup_longitude      1042806 non-null  float64            
 3   dropoff_latitude      1024316 non-null  float64            
 4   dropoff_longitude     1024316 non-null  float64            
 5   company               1122124 non-null  object             
 6   fare                  1122014 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(5), object(1)
memory usage: 59.9+ MB


In [11]:
df_for_model=df_for_model.dropna()
df_for_model.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1010127 entries, 1 to 1122123
Data columns (total 7 columns):
 #   Column                Non-Null Count    Dtype              
---  ------                --------------    -----              
 0   trip_start_timestamp  1010127 non-null  datetime64[ns, UTC]
 1   pickup_latitude       1010127 non-null  float64            
 2   pickup_longitude      1010127 non-null  float64            
 3   dropoff_latitude      1010127 non-null  float64            
 4   dropoff_longitude     1010127 non-null  float64            
 5   company               1010127 non-null  object             
 6   fare                  1010127 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(5), object(1)
memory usage: 61.7+ MB


Set Jupyter notbook to display all fields

In [12]:
pd.options.display.max_columns = None

Find if we have any records that have fare value of less than minimum taxi fare in Chicago, which is $2.70 and remove those rows as well 

In [13]:
df_for_model = df_for_model.loc[df_for_model['fare'] >= 2.70]
df_for_model.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1009336 entries, 1 to 1122123
Data columns (total 7 columns):
 #   Column                Non-Null Count    Dtype              
---  ------                --------------    -----              
 0   trip_start_timestamp  1009336 non-null  datetime64[ns, UTC]
 1   pickup_latitude       1009336 non-null  float64            
 2   pickup_longitude      1009336 non-null  float64            
 3   dropoff_latitude      1009336 non-null  float64            
 4   dropoff_longitude     1009336 non-null  float64            
 5   company               1009336 non-null  object             
 6   fare                  1009336 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(5), object(1)
memory usage: 61.6+ MB


Add new columns to indicate day of the week and hour of the day columns

In [14]:
df_for_model['trip_day_of_week']=df_for_model['trip_start_timestamp'].dt.dayofweek
df_for_model['trip_hour_of_day']=df_for_model['trip_start_timestamp'].dt.hour
df_for_model=df_for_model.drop(['trip_start_timestamp'],axis=1)
df_for_model.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1009336 entries, 1 to 1122123
Data columns (total 8 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   pickup_latitude    1009336 non-null  float64
 1   pickup_longitude   1009336 non-null  float64
 2   dropoff_latitude   1009336 non-null  float64
 3   dropoff_longitude  1009336 non-null  float64
 4   company            1009336 non-null  object 
 5   fare               1009336 non-null  float64
 6   trip_day_of_week   1009336 non-null  int64  
 7   trip_hour_of_day   1009336 non-null  int64  
dtypes: float64(5), int64(2), object(1)
memory usage: 69.3+ MB


Find the minum value for all columns

In [15]:
df_for_model.min()

pickup_latitude                         41.6601
pickup_longitude                       -87.9136
dropoff_latitude                        41.6601
dropoff_longitude                      -87.9136
company              2092 - 61288 Sbeih company
fare                                       2.85
trip_day_of_week                              0
trip_hour_of_day                              0
dtype: object

Find the maximum values for all columns

In [16]:
df_for_model.max()

pickup_latitude         42.016
pickup_longitude      -87.5349
dropoff_latitude       42.0212
dropoff_longitude     -87.5349
company              U Taxicab
fare                   9900.26
trip_day_of_week             6
trip_hour_of_day            23
dtype: object

Apply Min Max scalar to all numerical columns

In [17]:
scaler = MinMaxScaler(feature_range=(0, 1))
num_vars = ['pickup_latitude','pickup_longitude','dropoff_latitude','dropoff_longitude','fare','trip_day_of_week','trip_hour_of_day']
df_for_model[num_vars] = scaler.fit_transform(df_for_model[num_vars])
df_for_model.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1009336 entries, 1 to 1122123
Data columns (total 8 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   pickup_latitude    1009336 non-null  float64
 1   pickup_longitude   1009336 non-null  float64
 2   dropoff_latitude   1009336 non-null  float64
 3   dropoff_longitude  1009336 non-null  float64
 4   company            1009336 non-null  object 
 5   fare               1009336 non-null  float64
 6   trip_day_of_week   1009336 non-null  float64
 7   trip_hour_of_day   1009336 non-null  float64
dtypes: float64(7), object(1)
memory usage: 69.3+ MB


In [18]:
df_for_model.head()

,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,company,fare,trip_day_of_week,trip_hour_of_day
1,0.672827,0.740165,0.667625,0.626499,Taxi Affiliation Services,0.001000,0.833333,0.0
2,0.672827,0.740165,0.605752,0.761595,Taxi Affiliation Services,0.000470,0.833333,0.0
3,0.672827,0.740165,0.663180,0.740165,City Service,0.000596,0.833333,0.0
4,0.600908,0.660398,0.663180,0.740165,Taxi Affiliation Services,0.000520,0.833333,0.0
5,0.899463,0.000000,0.663180,0.740165,Sun Taxi,0.004057,0.833333,0.0


Gather the values that are used for scaling, so we can use those to decode our results during prediction

In [19]:
print("Note: Fare values were scaled by multiplying by {:.10f} and adding {:.6f}".format(scaler.scale_[4], scaler.min_[4]))
multiplied_by = scaler.scale_[4]
added = scaler.min_[4]

Note: Fare values were scaled by multiplying by 0.0001010365 and adding -0.000288


Company field is a categorical value, so lets add dummy variables

In [20]:
df_dummy = pd.get_dummies(df_for_model['company'],prefix='comp', drop_first=True)
df_for_model = pd.concat([df_for_model,df_dummy],axis=1)
df_for_model=df_for_model.drop('company',axis=1)
df_for_model.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1009336 entries, 1 to 1122123
Data columns (total 51 columns):
 #   Column                                             Non-Null Count    Dtype  
---  ------                                             --------------    -----  
 0   pickup_latitude                                    1009336 non-null  float64
 1   pickup_longitude                                   1009336 non-null  float64
 2   dropoff_latitude                                   1009336 non-null  float64
 3   dropoff_longitude                                  1009336 non-null  float64
 4   fare                                               1009336 non-null  float64
 5   trip_day_of_week                                   1009336 non-null  float64
 6   trip_hour_of_day                                   1009336 non-null  float64
 7   comp_24 Seven Taxi                                 1009336 non-null  uint8  
 8   comp_2733 - 74600 Benny Jona                       1009336 non

Split the data into training and test set

In [21]:
X=df_for_model.drop(['fare'],axis=1)
y=df_for_model['fare']

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.20,random_state=123)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(807468, 50)
(201868, 50)
(807468,)
(201868,)


Lets create a keras model and train using Train Data

In [23]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(50,)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1),
])

In [24]:
model.compile(optimizer='adam',
              loss='MeanSquaredError',
              metrics=['accuracy'])

In [25]:
hist = model.fit(X_train, y_train,
          batch_size=32, epochs=5,
          validation_data=(X_test, y_test))

Epoch 1/5
25234/25234 [==============================] - 37s 1ms/step - loss: 9.5963e-05 - accuracy: 0.0000e+00 - val_loss: 9.1193e-05 - val_accuracy: 4.9537e-06
Epoch 2/5
25234/25234 [==============================] - 35s 1ms/step - loss: 8.6044e-05 - accuracy: 0.0000e+00 - val_loss: 9.0134e-05 - val_accuracy: 4.9537e-06
Epoch 3/5
25234/25234 [==============================] - 34s 1ms/step - loss: 8.5108e-05 - accuracy: 0.0000e+00 - val_loss: 8.9237e-05 - val_accuracy: 4.9537e-06
Epoch 4/5
25234/25234 [==============================] - 34s 1ms/step - loss: 8.5155e-05 - accuracy: 0.0000e+00 - val_loss: 8.9449e-05 - val_accuracy: 4.9537e-06
Epoch 5/5
25234/25234 [==============================] - 34s 1ms/step - loss: 8.4856e-05 - accuracy: 0.0000e+00 - val_loss: 9.1925e-05 - val_accuracy: 4.9537e-06


Lets make prediction using the created model

In [26]:
prediction = model.predict(X_test[:1])
Expected_Fare = y_test.values[0]
print("Expected value with scaling - ",format(Expected_Fare))
Expected_Fare -= added
Expected_Fare /= multiplied_by
print("Expected Fare - ${}".format(Expected_Fare))

y_0 = prediction[0][0]
print("Prediction with scaling - ",format(y_0))
y_0 -= added
y_0 /= multiplied_by
print("Taxu_Fare Prediction  - ${}".format(y_0))

Expected value with scaling -  0.00034352421492087325
Expected Fare - $6.25
Prediction with scaling -  0.0008447542786598206
Taxu_Fare Prediction  - $11.210879445150495


Based on the above baseline, it seems LGBM is giving the best prediction.  Lets take that as our prototype